In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Lattice Boltzman Adveccion Difusion

# Python


In [164]:
import numpy as np
# Velocidad del lattice y velocidad del sonido
C = 1.0  # Velocidad característica del lattice
Cs = C / np.sqrt(3)  # Velocidad de la onda sonora
Cs2 = Cs * Cs  # Velocidad de la onda sonora al cuadrado

#-------------------------------VARIABLES GLOBALES------------------------

# Parámetros de difusión y relajación
D = 0.016  # Coeficiente de difusión
tau = (D / Cs2) + 0.5;  # Tiempo de relajación
Utau = 1.0 / tau  # Inverso del tiempo de relajación
UmUtau = 1 - Utau # 1 - 1/tau

#-------------------------------CLASES--------------------------------------
class LatticeBoltzman:
    # def int Lx, Ly, t_hour, iter_per_hour, Q
    # def double* w
    # def int index_f
    # def bint IsData
    # def int* Vx,
    # def int *Vy,


    # def public object f
    # def public object fnew

    # # Usamos object aquí en lugar de np.ndarray para evitar el error de buffer
    # def object id
    # def object rho_f
    # def object Ux
    # def object Uy

    def __init__(self, Lx, Ly, t_hour, iter_per_hour, id, rho_f,Ux, Uy,f):
        self.Lx = Lx
        self.Ly = Ly
        self.t_hour = t_hour
        self.iter_per_hour = iter_per_hour

        self.Q = 9
        self.w = np.zeros(self.Q, dtype=np.double)

        self.w[0] = 4.0 / 9
        for i in range(1, 5):
            self.w[i] = 1.0 / 9
        for i in range(5, 9):
            self.w[i] = 1.0 / 36

        self.Vx = np.zeros(self.Q, dtype=np.int64)  # Inicializar Vx como un arreglo de ceros
        self.Vx[8] = 1
        self.Vx[1] = 1
        self.Vx[5] = 1
        self.Vx[4] = 0
        self.Vx[0] = 0
        self.Vx[2] = 0
        self.Vx[7] = -1
        self.Vx[3] = -1
        self.Vx[6] = -1


        self.Vy = np.zeros(self.Q, dtype=np.int64)
        self.Vy[8] = -1
        self.Vy[1] = 0
        self.Vy[5] = 1
        self.Vy[4] = -1
        self.Vy[0] = 0
        self.Vy[2] = 1
        self.Vy[7] = -1
        self.Vy[3] = 0
        self.Vy[6] = 1


        self.Ux = Ux
        self.Uy = Uy
        self.id = id
        self.rho_f = rho_f


        # Asignación de los pesos y vectores de velocidad

        ArraySize = self.Lx * self.Ly * self.Q

        self.f = np.zeros(ArraySize, dtype=np.double)
        self.fnew = np.zeros(ArraySize, dtype=np.double)
        self.f = f

    def n(self, ix, iy, i):
        return (ix * self.Ly + iy) * self.Q + i

    def rho(self, ix, iy, UseNew):
        sum = 0.0
        for i in range(self.Q):
            n0 = self.n(ix, iy, i)
            if UseNew:
                sum += self.fnew[n0]
            else:
                sum += self.f[n0]
        return sum

    def load_f(self,  f):
      self.f = f

    def Jx(self, ix, iy, UseNew):
        sum = 0.0
        for i in range(self.Q):
            n0 = self.n(ix, iy, i)
            if UseNew:
                sum += self.Vx[i] * self.fnew[n0]
            else:
                sum += self.Vx[i] * self.f[n0]
        return sum

    def Jy(self, ix, iy, UseNew):
        sum = 0.0
        for i in range(self.Q):
            n0 = self.n(ix, iy, i)
            if UseNew:
                sum += self.Vy[i] * self.fnew[n0]
            else:
                sum += self.Vy[i] * self.f[n0]
        return sum

    def feq(self, rho0, Ux0, Uy0, i):
        UdotVi = Ux0 * self.Vx[i] + Uy0 * self.Vy[i]
        U2 = Ux0 * Ux0 + Uy0 * Uy0
        return rho0 * self.w[i] * (1 + UdotVi / Cs2 + (UdotVi * UdotVi) / (2.0 * Cs2 * Cs2) - U2 / (2.0 * Cs2))

    def Start(self, rho0, Ux0, Uy0):
        rho = rho0
        for ix in range(self.Lx):
            for iy in range(self.Ly):
                for i in range(self.Q):
                  n0 = self.n(ix, iy, i)
                  self.f[n0] = self.feq(rho, Ux0, Uy0, i)

    def Collision(self):
      for ix in range(self.Lx):
          for iy in range(self.Ly):
                rho0 = self.rho(ix, iy, False)
                Ux0 = self.Jx(ix, iy, False) / rho0
                Uy0 = self.Jy(ix, iy, False) / rho0
                for i in range(self.Q):
                    n0 = self.n(ix, iy, i)
                    self.fnew[n0] = UmUtau * self.f[n0] + Utau * self.feq(rho0, Ux0, Uy0, i)

    def ImposeFields(self, t):
        auxT = t //self.iter_per_hour
        index_tmp = 0

        for ix in range(self.Lx):
            for iy in range(self.Ly):
                index = ix * self.Ly + iy + self.Lx * self.Ly * auxT
                Ux0 = self.Ux[index]
                Uy0 = self.Uy[index]
                rho0 = self.rho(ix, iy, True)
                indexFire = ix * self.Ly + iy
                if ((indexFire) in self.id):
                    indexRho = np.where(self.id == indexFire)[0][0]
                    rho0=self.rho_f[indexRho]
                for i in range(self.Q):
                    n0 = self.n(ix, iy, i)
                    self.fnew[n0] = self.feq(rho0, Ux0, Uy0, i)

    def Advection(self):
        for ix in range(self.Lx):
            for iy in range(self.Ly):
                for i in range(self.Q):
                    ixnext = ix + self.Vx[i]
                    iynext = iy + self.Vy[i]
                    if 0 <= ixnext < self.Lx and 0 <= iynext < self.Ly:
                        n0 = self.n(ix, iy, i)
                        n0next = self.n(ixnext, iynext, i)
                        self.f[n0next] = self.fnew[n0]

    def Data(self):
        result = np.zeros((self.Lx, self.Ly), dtype=np.double)
        step = 1

        # Rellenar el array de resultados
        for ix in range(0, self.Lx, step):
            for iy in range(0, self.Ly, step):
                rho0 = self.rho(ix, iy, False)
                result[ix, iy] = rho0  # Asignar valores al array NumPy
        return result

    def printf(self):
        return self.f

In [81]:
import pandas as pd
#-------------------------------Carga de datos--------------------------------------

# Cargar datos de velocidad
data = np.loadtxt('/content/velocity.txt')
Ux = data[:, 0]
Uy = data[:, 1]
# Cargar datos de densidad de incendios
#data = np.loadtxt('/home/fire_coord.txt')
#fireId = data[:, 0].astype(int)
#rho_f = data[:, 1]

# Cargar los datos desde el archivo
data = np.loadtxt('/content/fire_coord.txt')

fireId = data[:, 0].astype(int)
rho_f = data[:, 1]

# Separar los datos usando -1 como delimitador
indices = np.where(fireId == -1)[0]  # Encontrar los índices donde hay -1
subarrays = np.split(fireId, indices)
fireId = [arr[arr != -1] for arr in subarrays]

indices = np.where(rho_f == -1)[0]  # Encontrar los índices donde hay -1
subarrays = np.split(rho_f, indices)
rho_f = [arr[arr != -1] for arr in subarrays]


#Parámetros de la simulación
data = np.loadtxt('/content/parameters.txt')
Lx = int(data[0])
Ly = int(data[1])

#Coordenadas estaciones
#data = ('/content/estaciones_coord.txt')
data = pd.read_csv('/content/estaciones_coord.txt')

data.drop(data.tail(1).index,inplace=True)

Est_ix= np.array(data['x_cell'])
Est_iy = np.array(data['y_cell'])

EstLin = Est_ix*Ly+Est_iy

numEstaciones = len(Est_ix)

#PM_estaciones = PM_base('/content/PM_base.csv')

# Leer el archivo CSV y extraer los valores numéricos
df = pd.read_csv('/content/PM_base.csv', header=None)

array = df[0].to_numpy()

# Partir el arreglo en subarreglos de longitud 14
PM_estaciones = array.reshape(-1, numEstaciones)


In [88]:
#-------------------------------SIMULACIÓN--------------------------------------
def simulation(Lx, Ly, t_hour,  t_inital, t_final, iter_per_hour, Ux, Uy, id, rho_f):
    rho0 = 0.001
    Ux0 = 0.0
    Uy0 = 0.0

     # Parámetros para la distribución gaussiana que inicializa la densidad
    mu_x = Lx / 2.0
    mu_y = Ly / 2.0
    sigma_x = Lx / 32.0
    sigma_y = Ly / 32.0

    lbm = LatticeBoltzman(Lx, Ly, t_hour, iter_per_hour, id, rho_f, Ux, Uy)
    lbm.Start(rho0, Ux0, Uy0)

    # Simular por 100 pasos de tiempo
    for t in range(t_inital, t_final):
        lbm.Collision()
        lbm.ImposeFields(t)
        lbm.Advection()

    return  lbm.Data()

In [92]:
#-------------------------------MAIN--------------------------------------
Lx = 100
Ly = int(Lx*1.4)
t_hour = 100
iter_per_hour = 10
t_inital = 0
t_final = t_hour*iter_per_hour

size = 10

id = np.array([123,33,65])
rho_f = np.array([4,4,4])


# Ejecutar la simulación
result_simulation = simulation(Lx, Ly, t_hour, t_inital,t_final, iter_per_hour, Ux, Uy, id, rho_f)
result_simulation.shape

IndexError: index 33 is out of bounds for axis 0 with size 3

## Simulacion

In [131]:
#-------------------------------Inicio_SIMULACIÓN--------------------------------------
def simulationStart(id, rho_f, Ux, Uy):

    Lx=100
    Ly=140
    t_hour=240
    iter_per_hour=9604
    rho0=0.001
    Ux0=0
    Uy0=0
    Q=9
    lbm = LatticeBoltzman(Lx, Ly, t_hour, iter_per_hour, id, rho_f, Ux, Uy, np.zeros(Lx*Ly*Q))
    lbm.Start(rho0, Ux0, Uy0)
    return  lbm.printf()



In [172]:
#-------------------------------SIMULATION--------------------------------------
def simulation(t_inital, t_final, id, rho_f, Ux, Uy,fDist):

    Lx=100
    Ly=140
    t_hour=240
    iter_per_hour=9604

    lbm = LatticeBoltzman(Lx, Ly, t_hour, iter_per_hour, id, rho_f, Ux, Uy,fDist)
    #lbm.loadf(fDist)

    # Simular por 100 pasos de tiempo
    for t in range(int(t_inital), int(t_final)):
        lbm.Collision()
        lbm.ImposeFields(t)
        lbm.Advection()
        print("Porcentaje Avance sin ajuste: ", 100*t/(t_final-t_inital)," %")

    return  lbm.printf()


In [122]:
#-------------------------------AJUSTE--------------------------------------
def simulationAjuste(t_inital, t_final, id, rho_f, Ux, Uy,fDist):

    Lx=100
    Ly=140
    t_hour=240
    iter_per_hour=9604
    lbm = LatticeBoltzman(Lx, Ly, t_hour, iter_per_hour, id, rho_f, Ux, Uy,fDist)
    #lbm.loadf(fDist)
    # Simular por 100 pasos de tiempo
    for t in range(int(t_inital), int(t_final)):
        lbm.Collision()
        lbm.ImposeFields(t)
        lbm.Advection()

    return  lbm.printf()



## Carga de Datos

In [179]:
import pandas as pd
#-------------------------------Carga de datos--------------------------------------

# Cargar datos de velocidad
data = np.loadtxt('/content/velocity.txt')
Ux = data[:, 0]
Uy = data[:, 1]
# Cargar datos de densidad de incendios
#data = np.loadtxt('/home/fire_coord.txt')
#fireId = data[:, 0].astype(int)
#rho_f = data[:, 1]

# Cargar los datos desde el archivo
data = np.loadtxt('/content/fire_coord.txt')

fireId = data[:, 0].astype(int)
rho_f = data[:, 1]

# Separar los datos usando -1 como delimitador
indices = np.where(fireId == -1)[0]  # Encontrar los índices donde hay -1
subarrays = np.split(fireId, indices)
fireId = [arr[arr != -1] for arr in subarrays]

indices = np.where(rho_f == -1)[0]  # Encontrar los índices donde hay -1
subarrays = np.split(rho_f, indices)
rho_f = [arr[arr != -1] for arr in subarrays]


#Parámetros de la simulación
data = np.loadtxt('/content/parameters.txt')
Lx = int(data[0])
Ly = int(data[1])

#Coordenadas estaciones
#data = ('/content/estaciones_coord.txt')
data = pd.read_csv('/content/estaciones_coord.txt')

data.drop(data.tail(1).index,inplace=True)

Est_ix= np.array(data['x_cell'])
Est_iy = np.array(data['y_cell'])

EstLin = Est_ix*Ly+Est_iy

numEstaciones = len(Est_ix)

#PM_estaciones = PM_base('/content/PM_base.csv')

# Leer el archivo CSV y extraer los valores numéricos
df = pd.read_csv('/content/PM_base.csv', header=None)

array = df[0].to_numpy()

# Partir el arreglo en subarreglos de longitud 14
PM_estaciones = array.reshape(-1, numEstaciones)


# Ajuste

In [175]:
def objective(rho_sources_opt, stations, density_observed_stations, sources_1D,fDist,t,Ux,Uy):
    """
    Calcula el error entre la densidad observada y la densidad simulada solo en las celdas que tienen estaciones.

    Parameters:
    - rho_sources_opt: valores optimizados de rho para las celdas fuente.
    - stations: lista de índices de celdas con estaciones.
    - density_observed: arreglo unidimensional de densidad observada.
    Returns:
    - error: suma de los cuadrados de las diferencias en las celdas de las estaciones.
    """
    # Recalcular la densidad simulada usando los valores optimizados de rho
    simulated_stations_opt = simulationAjuste(t,t+1,sources_1D,rho_sources_opt,Ux,Uy,fDist)

    # Extraer las densidades en las estaciones de la densidad observada y simulada optimizada
    simulated_stations_opt = simulated_stations_opt[stations]

    print(f'{density_observed_stations=}')
    print(f'{simulated_stations_opt=}')
    # Calcular el error cuadrático medio entre las densidades observadas y simuladas en las estaciones
    error = np.sum((density_observed_stations - simulated_stations_opt) ** 2)
    print("error", error)
    return error

def optimize_parameters_for_cells(stations, density_observed, sources_1D, rho_sources,fDist,t,Ux,Uy):
    """Optimiza los valores de rho para las celdas fuente para minimizar el error entre densidad simulada y observada en las estaciones."""
    rho_initial = rho_sources  # Valores iniciales para las celdas fuente
    bounds = [(0, 8e7)] * len(sources_1D)  # Restricciones sobre los valores de las celdas fuente

    # Ejecutar la optimización
    result = minimize(objective, rho_initial, args=(stations, density_observed, sources_1D,fDist,t,Ux,Uy), method='L-BFGS-B', bounds=bounds,tol=0.1,options={'iprint': 99})

    # Resultados óptimos
    rho_opt = result.x
    return rho_opt

In [ ]:
from scipy.optimize import minimize



# Ejemplo de uso

#sources_1D = []  # Lectura ubicación de todas las fuentes para todas las horas.
#initial_rho_sources = [] #Valores iniciales de rho de las fuentes para todas las horas.
#velocityData = [] #Valores de velocidades para todas las horas.

#stations = obs_data.coord_est('../../Data/estaciones_coord.txt') #Ubicación de las estaciones. #Partido por días
#PM_estaciones = obs_data.PM_base('../../Data/PM_base.csv') #Datos de contaminación en estaciones. #Partido por días

iter_per_hour = 9604
#timesAdjusment = np.arange(0,9604*24,9604/2) #Lectura de todos los tiempos para ajuste.
timesAdjusment = np.arange(0,9604*2,9604/2) #Lectura de todos los tiempos para ajuste.

#timesAdjusment[0,9604/4,(9604/4)*2]

#######################################################

indexFuegoHora = fireId[0]
rhoFuegoInicialHora = rho_f[0]
fDist = simulationStart(indexFuegoHora, rhoFuegoInicialHora,Ux,Uy) #Genera un archivo de fnew para todas las celdas linealizadas.

#f  -> Colisiona, Impone campos(Ajuste), Advecciona (Rho_simulado=f)
#Se imprime la función de equilibrio solo cuando t=timesAdjustment
#Load: Cargar las f
#


with open('OutputInfo.txt', 'a') as f:

  for i, t in enumerate(timesAdjusment):
      print(f"Ajuste {i} de {len(timesAdjusment)}")
      # Optimización de los parámetros de las fuentes
      tInicial = timesAdjusment[i]
      if(i != len(timesAdjusment)):
        tFinal = timesAdjusment[i+1]
      else:
        break

      estacionesHora = EstLin

      indexHora = int((t+1)//(iter_per_hour*24))
      valoresEstacionesHora = PM_estaciones[indexHora]
      indexFuegoHora = fireId[indexHora]
      rhoFuegoInicialHora = rho_f[indexHora]
      rho_opt = optimize_parameters_for_cells(estacionesHora, valoresEstacionesHora, indexFuegoHora,rhoFuegoInicialHora,fDist,t,Ux,Uy)
      rho_opt=rhoFuegoInicialHora
      print(f'{rhoFuegoInicialHora=}')
      print(f'{rho_opt=}')
      print(f'{indexFuegoHora=}')
      data = np.column_stack((rho_opt,indexFuegoHora))
      np.savetxt(f,data, delimiter=' ', fmt='%d',newline='\n')
      f.write('\n')
      fDist = simulation(t_inital, t_final, indexFuegoHora, rhoFuegoInicialHora, Ux, Uy,fDist)






Ajuste 0 de 4
density_observed_stations=array([ 1.0941048, 54.70524  ,  1.0941048,  1.0941048,  1.0941048,
        1.0941048,  0.       ,  0.       ,  0.       , 21.882096 ,
       61.2698688,  1.0941048,  0.       ,  1.0941048])
simulated_stations_opt=array([2.79724210e-05, 2.80101883e-05, 2.78189403e-05, 2.78189403e-05,
       4.44427442e-04, 1.10843839e-04, 2.77364403e-05, 2.81287513e-05,
       2.78189403e-05, 2.75451883e-05, 2.79647019e-05, 2.75918685e-05,
       1.10873892e-04, 4.44442555e-04])
error 7233.85556139574
density_observed_stations=array([ 1.0941048, 54.70524  ,  1.0941048,  1.0941048,  1.0941048,
        1.0941048,  0.       ,  0.       ,  0.       , 21.882096 ,
       61.2698688,  1.0941048,  0.       ,  1.0941048])
simulated_stations_opt=array([2.79724210e-05, 2.80101883e-05, 2.78189403e-05, 2.78189403e-05,
       4.44552211e-04, 1.10843839e-04, 2.77364403e-05, 2.81287513e-05,
       2.78189403e-05, 2.75451883e-05, 2.79647019e-05, 2.75918685e-05,
       1.10873892e-